In [12]:
import tensorflow;
import os;

os.environ["TL_BACKEND"] = "tensorflow";
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async";

import tensorlayerx;
from tensorflow import convert_to_tensor;
import numpy;

from tensorflow.keras.datasets import mnist;

In [13]:
print(tensorflow.config.list_physical_devices('GPU'));


import tensorflow as tf

# Set memory growth for GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        # Set memory growth
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("Memory growth set for GPU")
    except RuntimeError as e:
        print(e)

[]


In [14]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data();

# Split the test and val by 50:50
test_val_images_split = numpy.array_split(test_images, 2);
test_val_labels_split = numpy.array_split(test_labels, 2);

test_images = test_val_images_split[0];
test_labels = test_val_labels_split[0];

val_images = test_val_images_split[1];
val_labels = test_val_labels_split[1];

In [15]:
# Recall step 1
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255;
test_images = test_images.reshape((5000, 28, 28, 1)).astype('float32') / 255;
val_images = val_images.reshape((5000, 28, 28, 1)).astype('float32') / 255;


# Recall step 2
train_labels = tensorflow.one_hot(train_labels, depth = 10);
test_labels = tensorflow.one_hot(test_labels, depth = 10);
val_labels = tensorflow.one_hot(val_labels, depth = 10);

In [20]:
from tensorlayerx.nn import Conv2d, MaxPool2d, Flatten, Linear, Module, Input, ReLU, Softmax, Sequential;

In [21]:
class CNNModel (Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        self.conv1 = Conv2d(out_channels = 32, kernel_size = (3, 3), stride = (1, 1), in_channels = 1, act = ReLU, name = "conv1");
        self.pool1 = MaxPool2d(kernel_size = (2, 2), name = "pool1");

        self.conv2 = Conv2d(out_channels = 64, kernel_size = (1, 1), stride = (1, 1), in_channels = 1, act = ReLU, name = "conv2");
        self.pool2 = MaxPool2d(kernel_size = (2, 2), name = "pool2");

        self.flat = Flatten(name = "flat");

        self.dense = Linear(out_features = 64, in_features = 50176, act = ReLU, name = "dense1");
        self.output = Linear(out_features = 10, in_features = 64, act = Softmax, name = "output")
    
    def compile(self):
        model = Sequential();
        model.add(self.conv1);
        model.add(self.pool1);
        model.add(self.conv2);
        model.add(self.pool2);
        model.add(self.flat);
        model.add(self.dense);
        model.add(self.output);

        print("------------------------------------------------")
        print("             MNIST Tensorlayerx [TF]            ");
        print("------------------------------------------------")
        model.summary();

        return model;

In [22]:
CNNModel().compile();

[TLX] Conv2d conv1: out_channels : 32 kernel_size: (3, 3) stride: (1, 1) pad: SAME act: ReLU
[TLX] MaxPool2d pool1: kernel_size: (2, 2) stride: (2, 2) padding: SAME return_mask: False
[TLX] Conv2d conv2: out_channels : 64 kernel_size: (1, 1) stride: (1, 1) pad: SAME act: ReLU
[TLX] MaxPool2d pool2: kernel_size: (2, 2) stride: (2, 2) padding: SAME return_mask: False
[TLX] Flatten flat:
[TLX] Linear  dense1: 64 ReLU
[TLX] Linear  output: 10 Softmax


AttributeError: 'Sequential' object has no attribute 'add'.